In [1]:
import boto3
import json

In [ ]:
!mkdir ~/.aws

In [13]:

!echo [default]  > ~/.aws/credentials
!echo aws_access_key_id=xxxxxxxxxxxxxxxxxxxxx >> ~/.aws/credentials
!echo aws_secret_access_key=xxxxxxxxxxxxxxxxxxxxxxxx >> ~/.aws/credentials
!echo aws_session_token=xxxxxxxxxxxx >> ~/.aws/credentials


In [14]:
!cat ~/.aws/credentials

[default]
aws_access_key_id=xxxxxxxxxxxxxxxxxxxxx
aws_secret_access_key=xxxxxxxxxxxxxxxxxxxxxxxx
aws_session_token=xxxxxxxxxxxx


In [16]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')
try:
    response = dynamodb_client.describe_table(TableName='test')
except dynamodb_client.exceptions.ResourceNotFoundException:
#     do nothing - tabe does not exist
    print('Table does not exist')
else:
    print('Table does exists, deleting it')
    dynamodb.Table('test').delete()

Table does exists, deleting it


In [4]:
table = dynamodb.create_table(
    TableName='test',
    KeySchema=[
        {
            'AttributeName': 'author',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'title_date',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'author',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'title_date',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 100,
        'WriteCapacityUnits': 100,
    }
)

table.meta.client.get_waiter('table_exists').wait(TableName='test')
print(table.item_count)

0


In [5]:
from decimal import Decimal
table = dynamodb.Table('test')
count = 0
with open('nyt2.json', 'r') as f:
    books_dict = json.load(f)
for book in books_dict: 
    _id = str(book['_id'])
    bestsellers_date = str(book['bestsellers_date'])
    published_date = str(book['published_date'])
    amazon_product_url = str(book['amazon_product_url'])
    author = str(book['author'])
    description = str(book['description'])
    #print(_id)
    price = Decimal(book['price'])
    publisher = str(book['publisher'])
    title_date = str(book['title'])+'#'+str(book['published_date'])
    rank = int(book['rank'])
    rank_last_week = int(book['rank_last_week'])
    weeks_on_list = int(book['weeks_on_list'])
    #print(price)
    
    table.put_item(
        Item={
           '_id' : _id, 
           'bestsellers_date' : bestsellers_date,
           'published_date' : published_date,
           'amazon_product_url' : amazon_product_url,
           'author' : author,          

           'price' : price,
           'publisher' : publisher,
           'title_date' : title_date,
           'rank' : rank,
           'rank_last_week' : rank_last_week,
           'weeks_on_list' : weeks_on_list
        }
    )
    count += 1
    if count % 1000 == 0:
        print(count, book['_id'])
#eof
print('Read ', count, ' lines, last line: ', book['_id'], ', table count: ', table.item_count) 

1000 5b4aa4ead3089013507db572
2000 5b4aa4ead3089013507db95a
3000 5b4aa4ead3089013507dbd42
4000 5b4aa4ead3089013507dc12a
5000 5b4aa4ead3089013507dc512
6000 5b4aa4ead3089013507dc8fa
7000 5b4aa4ead3089013507dcce2
8000 5b4aa4ead3089013507dd0ca
9000 5b4aa4ead3089013507dd4b2
10000 5b4aa4ead3089013507dd89a
Read  10195  lines, last line:  5b4aa4ead3089013507dd95d , table count:  0


## Example: Show all rows where the author is John Sandford

In [11]:
from boto3.dynamodb.conditions import Key,Attr
table = dynamodb.Table('test')
response = table.query(
  KeyConditionExpression=Key('author').eq('James Ellroy')
)
print(len(response['Items']))

0